In [19]:
import pandas as pd
import streamlit as st
import requests
from requests import post, get
import base64
import urllib
import os
import json
from dotenv import load_dotenv
import plotly.express as px

In [36]:
data= {'code':['AR','US','ES'],
       'country':['Argentina','United States','Spain'],
       'capital':['Buenos Aires', 'Washington','Madrid'],
       'lat':[-34.603722, 38.889805, 40.416775],
       'long': [-58.381592, -77.009056,-3.703790]}

In [37]:
df= pd.DataFrame(data)
df

,code,country,capital,lat,long
0,AR,Argentina,Buenos Aires,-34.603722,-58.381592
1,US,United States,Washington,38.889805,-77.009056
2,ES,Spain,Madrid,40.416775,-3.703790


In [38]:
def function(input):
    return input +'modif'

In [39]:
df['new_column']= [function(df.loc[index,'code']) for index in range(len(df))]
df

,code,country,capital,lat,long,new_column
0,AR,Argentina,Buenos Aires,-34.603722,-58.381592,ARmodif
1,US,United States,Washington,38.889805,-77.009056,USmodif
2,ES,Spain,Madrid,40.416775,-3.703790,ESmodif


In [2]:
# Spotify Developer Dashboard details
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# We first request a token to the Spotify Accounts Service, which will be used later on to access the Spotify Web API
# To get our access token, we need to pass our client ID, client Secret and grant_type
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    # Spotify requests to encode the client ID and client secret using base64
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    # Make the request
    result = post(url,headers= headers, data= data)
    # Convert the result (json) into a python dictionary
    json_result = json.loads(result.content) 
    token = json_result["access_token"]
    return token

<class 'str'>


In [11]:
token= get_token()

In [12]:
# Function to construct the header to send a request
def get_auth_header(token):
    return{"Authorization": "Bearer " + token}

In [13]:
# Get new releases
def get_new_releases(country):
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['name']
    return json_result

In [14]:
new_releases= get_new_releases("AR")

In [40]:
df['new_releases']= [get_new_releases(df.loc[index,'code']) for index in range(len(df))]
df

,code,country,capital,lat,long,new_column,new_releases
0,AR,Argentina,Buenos Aires,-34.603722,-58.381592,ARmodif,"Peso Pluma: Bzrp Music Sessions, Vol. 55"
1,US,United States,Washington,38.889805,-77.009056,USmodif,Space Heavy
2,ES,Spain,Madrid,40.416775,-3.703790,ESmodif,MAÑANA SERÁ BONITO


In [41]:
fig = px.scatter_geo(df, lat='lat', lon= 'long',
                     text= 'country', hover_name='new_releases', color='country'
                     )
fig.show()

In [20]:
px.scatter_geo?

Signature:
px.scatter_geo(
    data_frame=None,
    lat=None,
    lon=None,
    locations=None,
    locationmode=None,
    geojson=None,
    featureidkey=None,
    color=None,
    text=None,
    symbol=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    size=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    symbol_sequence=None,
    symbol_map=None,
    opacity=None,
    size_max=None,
    projection=None,
    scope=None,
    center=None,
    fitbounds=None,
    basemap_visible=None,
    title=None,
    template=None,
    width=None,
    height=None,
) -> plotly.graph_objs._figure.Figure
Docstring:
    In a geographic scatter plot, each row of